In [1]:
import pandas as pd

path = r'/home/dingw/work/CleanedDATA V12-05-2021.csv'
df = pd.read_csv(path, sep=';', encoding='windows-1252')

series = df['Publication identifier,,,,,,,,,,,,,,,,,,']
for i in range(len(series)):
    j_comma = series[i].find(',')
    if j_comma > 0:
        series[i] = series[i][:j_comma]
    
df.rename(columns={'Publication identifier,,,,,,,,,,,,,,,,,,': 'Publication identifier'}, inplace=True)
# df.to_csv(path, sep=';', index=False, encoding='windows-1252')

/tmp/ipykernel_28805/2095857185.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series[i] = series[i][:j_comma]


In [6]:
from utils.similarity import *
# attribute_name, value, sim_func, weight
# ---------------------------------------
class Case:
    attr_func = {'Task': sim_task, 'Case study type': sim_case_study_type, 'Case study': Levenshtein, 
                 'Online/Off-line': sim_on_offline, 'Input for the model': Levenshtein
                 }

    def __init__(self, *args, **kwargs):
        attributes = [None] * len(self.attr_func)
        for i, attr in enumerate(self.attr_func.keys()):
            if i < len(args):
                attributes[i] = args[i]
            if attr in kwargs:
                attributes[i] = kwargs[attr]
        
        self.attributes = attributes

    @classmethod
    def from_series(cls, series):
        return cls(**series.to_dict())
    

    def cal_similarity(self, case, weights=[1.0]*len(attr_func)):
        assert isinstance(case, Case)
        assert len(weights) == len(self.attr_func)

        sims = []
        for i, (attr, func) in enumerate(self.attr_func.items()):
            sims.append(func(self.attributes[i], case.attributes[i]))
        global_sim = sum([sim * weight for sim, weight in zip(sims, weights)]) / sum(weights)
        return global_sim
    
s = df.loc[0,['Task', 'Case study type', 'Case study', 'Online/Off-line', 'Input for the model']]
c1 = Case.from_series(s)
s = df.loc[1,['Task', 'Case study type', 'Case study', 'Online/Off-line', 'Input for the model']]
c2 = Case.from_series(s)

print(c1.cal_similarity(c2, [1, 1, 1, 1, 1]))
print(c2.cal_similarity(c1, [1, 1, 1, 1, 1]))
print(c1.cal_similarity(c2, [1, 0, 0, 0, 0]))
print(c2.cal_similarity(c2, [1, 1, 1, 1, 1]))

0.9800000000000001
0.9800000000000001
0.9
1.0


In [3]:
c2.attributes

['Health assessment',
 'Rotary machines',
 'Simulated jet-engines data',
 'Off-line',
 'Time series']